In [12]:
import pandas as pd
pd.set_option('max_columns', None)
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from nltk import metrics, stem, tokenize
import nltk
import glob

In [3]:
one_race_cols = {"Driver": "Driver", "Car": "Car #", "AvgStart": "Start", "AvgMidRace": "Mid Race", "AvgFinish": "Finish",
                 "Avg Pos": "Average Position", "PassDiff": "Pass Differential", "GreenPass": "Green Passes",
                 "GreenPassed": "Green Passed", "QualityPasses": "Quality Passes",
                 "PercentQualityPasses": "% Quality Passes", "NumFastestLaps": "# Fastest Laps",
                 "LapsinTop15": "Laps in Top 15", "PercentLapsTop15": "% Laps in Top 15", "LapsLed": "Laps Led",
                 "PercentLapsLed": "% Laps Led", "TotalLaps": "Total Laps", "DriverRating": "Driver Rating",
                 "Pts": "Points"}

one_race_cols_2 = {"Driver": "Driver", "Car": "Car #", "AvgPos": "Average Position", "PassDiff": "Pass Differential",
                   "GreenPass": "Green Passes", "GreenPassed": "Green Passed", "QualityPasses": "Quality Passes",
                   "PercentQualityPasses": "% Quality Passes", "NumFastestLaps": "# Fastest Laps",
                   "LapsTop15": "Laps in Top 15", "PercentLapsTop15": "% Laps in Top 15", "LapsLed": "Laps Led",
                   "PercentLapsLed": "% Laps Led", "TotalLaps": "Total Laps", "DriverRating": "Driver Rating",
                   "Pts": "Points", "StartPos": "Start", "MidRace": "Mid Race", "FinishPos": "Finish"}

car_nums_22 = {"AJ Allmendinger": 16, "Aric Almirola": 10, "Christopher Bell": 20, "Greg Biffle": 44,
             "Josh Bilicki": 77, "Ryan Blaney": 12, "Alex Bowman": 48, "Chase Briscoe": 14,
             "Chris Buescher": 17, "Harrison Burton": 21, "Kurt Busch": 45, "Kyle Busch": 18,
             "William Byron": 24, "Landon Cassill": 77, "Ross Chastain": 1,
             "Austin Cindric": 2, "Cole Custer": 41, "Austin Dillon": 3, "Ty Dillon": 42,
             "Chase Elliott": 9, "Todd Gilliland": 38, "Noah Gragson": 16, "Kaz Grala": 50,
             "Denny Hamlin": 11, "Joey Hand": 15, "Kevin Harvick": 4, "Justin Haley": 31,
             "Daniel Hemric": 16, "Loris Hezemans": 27, "Erik Jones": 20, "Brad Keselowski": 6,
             "Corey LaJoie": 7, "Kyle Larson": 5, "Joey Logano": 22, "BJ McLeod": 78,
             "Michael McDowell": 34, "Ryan Preece": 15, "David Ragan": 15, "Tyler Reddick": 8,
             "Boris Said": 66, "Garrett Smithley": 15, "Ricky Stenhouse Jr.": 47, "Daniel Suárez": 99,
             "Martin Truex Jr.": 19, "Jacques Villeneuve": 27, "Bubba Wallace": 23, "Cody Ware": 51}

teams_22 = {"AJ Allmendinger": "Kaulig", "Aric Almirola": "SHR", "Christopher Bell": "JGR", "Greg Biffle": "NYR",
        "Josh Bilicki": "Spire", "Ryan Blaney": "Penske", "Alex Bowman": "Hendrick", "Chase Briscoe": "SHR",
        "Chris Buescher": "RFK", "Harrison Burton": "WBR", "Kurt Busch": "23X1", "Kyle Busch": "JGR",
        "William Byron": "Hendrick", "Landon Cassill": "Spire", "Ross Chastain": "Trackhouse", "Austin Cindric": "Penske",
        "Cole Custer": "SHR", "Austin Dillon": "RCR", "Ty Dillon": "PGMS", "Chase Elliott": "Hendrick",
        "Todd Gilliland": "FRM", "Noah Gragson": "Kaulig", "Kaz Grala": "TMT", "Denny Hamlin": "JGR",
        "Joey Hand": "RWR", "Kevin Harvick": "SHR", "Justin Haley": "Kaulig", "Daniel Hemric": "Kaulig",
        "Loris Hezemans": "Hezeberg", "Erik Jones": "PGMS", "Brad Keselowski": "RFK", "Corey LaJoie": "Spire",
        "Kyle Larson": "Hendrick", "Joey Logano": "Penske", "BJ McLeod": "LFM", "Michael McDowell": "FRM",
        "Ryan Preece": "RWR", "David Ragan": "RWR", "Tyler Reddick": "RCR", "Boris Said": "MBM",
        "Garrett Smithley": "RWR", "Ricky Stenhouse Jr.": "JTGD", "Daniel Suárez": "Trackhouse", "Martin Truex Jr.": "JGR",
        "Jacques Villeneuve": "Hezeberg", "Bubba Wallace": "23X1", "Cody Ware": "RWR"}

makes_22 = {"23X1": "Toyota", "Beard": "Chevy", "FRM": "Ford", "Hendrick": "Chevy",
        "Hezeberg": "Ford", "JGR": "Toyota", "JTGD": "Chevy", "Kaulig": "Chevy",
        "LFM": "Ford", "MBM": "Ford", "NYR": "Chevy", "Penske": "Ford",
        "PGMS": "Chevy", "RCR": "Chevy", "RFK": "Ford", "RWR": "Ford",
        "SHR": "Ford", "Spire": "Chevy", "TMT": "Chevy", "Trackhouse": "Chevy",
        "WBR": "Ford"}

track_types_22 = {"Auto Club": "Speedway", "Atlanta": "Superpeedway", "Bristol": "Short Track", "COTA": "Road Course",
               "Daytona": "Superspeedway", "Las Vegas": "Speedway", "Martinsville": "Short Track", "Phoenix": "Speedway",
               "Richmond": "Short Track", "Talladega": "Superspeedway", "Dover": "Short Track", "Darlington": "Speedway"}

laps_22 = {"Daytona1": 201, "Auto Club": 200, "Las Vegas1": 274, "Phoenix1": 312, "Atlanta1": 325,
           "COTA": 69, "Richmond1": 400, "Martinsville1": 403, "Bristol Dirt": 250, "Talladega1": 188,
           "Dover": 400, "Darlington1": 999}

laps_racenum_22 = {"Daytona1": 1, "Auto Club": 2, "Las Vegas1": 3, "Phoenix1": 4, "Atlanta1": 5,
           "COTA": 6, "Richmond1": 7, "Martinsville1": 8, "Bristol Dirt": 9, "Talladega1": 10,
           "Dover": 11, "Darlington1": 12}

In [49]:
# reading in 2022 data
import re
from nltk import metrics, stem, tokenize
def normalize(s):
    stemmer = stem.PorterStemmer()
    words = tokenize.wordpunct_tokenize(s.lower().strip())
    return ' '.join([stemmer.stem(w) for w in words])
 
def fuzzy_match(s1, s2, max_dist=3):
    return nltk.edit_distance(normalize(s1), normalize(s2)) <= max_dist

def add_cols_22(df, race, track = ""):
    df["Driver-Car"] = df["Driver"] + "_" + df["Car #"].astype(str)
    df["Points Eligible"] = [1 if row > 0 else 0 for row in df["Points"]]
    df["Team"] = df["Driver"].map(teams_22)
    df["Make"] = df["Team"].map(makes_22)
    # df["Race #"] = racenum
    df["Track"] = track
    df["Track Type"] = df["Track"].map(track_types_22)
    df["Race Laps"] = laps_22[race]
    df["% Fastest Laps"] = round(df["# Fastest Laps"] / df["Race Laps"] * 100, 3)
    df["Pass Diff Per Lap"] = round(df["Pass Differential"] / df["Total Laps"], 3)
    df["Quality Passes Per Lap"] = round(df["Quality Passes"] / df["Total Laps"], 3)
    return df

data_path = r'C:\Users\kchen\OneDrive\MADS\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\csv\2022'

filenames = glob.glob(data_path + "/*.csv")
loop_data=[]

for file in filenames:
    df=pd.read_csv(file)
    race_date=re.search(r'(\d{8})_(\w*)',file)
    date=race_date.group(1)
    race=race_date.group(2)
    df['race']=race
    df['date']=date
    df["Driver-Car"] = df["Driver"] + "_" + df["Car"].astype(str)
    df["Points Eligible"] = [1 if row > 0 else 0 for row in df["Pts"]]
    df["Team"] = df["Driver"].map(teams_22)
    df["Make"] = df["Team"].map(makes_22)
    df["% Fastest Laps"] = round(df["NumFastestLaps"] / df["TotalLaps"] * 100, 3)
    df["Pass Diff Per Lap"] = round(df["PassDiff"] / df["TotalLaps"], 3)
    df["Quality Passes Per Lap"] = round(df["QualityPasses"] / df["TotalLaps"], 3)
    loop_data.append(df)

pd.concat(loop_data)

,Car,Driver,StartPos,MidRace,CloserPos,FinishPos,HighPos,LowPos,AvgPos,PassDiff,GreenPass,GreenPassed,QualityPasses,PercentQualityPasses,NumFastestLaps,LapsTop15,PercentLapsTop15,LapsLed,PercentLapsLed,TotalLaps,DriverRating,Pts,race,date,Driver-Car,Points Eligible,Team,Make,% Fastest Laps,Pass Diff Per Lap,Quality Passes Per Lap
0,2,Austin Cindric,5,2,9999,1,9999,9999,6.2,-13,168,181,125,74.40,0,176,87.6,21,10.4,201,122.4,54,Daytona500_modified,20220220,Austin Cindric_2,1,Penske,Ford,0.000,-0.065,0.622
1,6,Brad Keselowski,3,5,9999,9,9999,9999,4.8,1,171,170,148,86.50,4,198,98.5,67,33.3,201,121.3,54,Daytona500_modified,20220220,Brad Keselowski_6,1,RFK,Ford,1.990,0.005,0.736
2,19,Martin Truex Jr,14,18,9999,13,9999,9999,11.9,25,258,233,192,74.40,4,143,71.1,11,5.5,201,94.0,49,Daytona500_modified,20220220,Martin Truex Jr_19,1,NaN,NaN,1.990,0.124,0.955
3,23,Bubba Wallace,16,19,9999,2,9999,9999,10.2,16,253,237,191,75.50,7,145,72.1,12,6.0,201,104.7,47,Daytona500_modified,20220220,Bubba Wallace_23,1,23X1,Toyota,3.483,0.080,0.950
4,14,Chase Briscoe,9,10,9999,3,9999,9999,18.3,17,209,192,70,33.50,6,88,43.8,0,0.0,201,75.0,41,Daytona500_modified,20220220,Chase Briscoe_14,1,SHR,Ford,2.985,0.085,0.348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,77,Josh Bilicki,36,34,32,32,32,36,34.1,-3,1,4,0,0.00,0,0,0.0,0,0.0,381,26.7,0,DoverMotorSpeedway,20220502,Josh Bilicki_77,0,Spire,Chevy,0.000,-0.008,0.000
32,16,AJ Allmendinger,19,2,33,33,2,31,18.9,-21,47,68,18,38.30,4,179,44.8,0,0.0,321,62.6,0,DoverMotorSpeedway,20220502,AJ Allmendinger_16,0,Kaulig,Chevy,1.246,-0.065,0.056
33,51,Cody Ware,28,33,34,34,26,33,32.9,-3,11,14,0,0.00,0,0,0.0,0,0.0,238,31.8,3,DoverMotorSpeedway,20220502,Cody Ware_51,1,RWR,Ford,0.000,-0.013,0.000
34,78,BJ McLeod,32,35,35,35,32,36,34.7,3,5,2,0,0.00,0,0,0.0,0,0.0,167,24.8,2,DoverMotorSpeedway,20220502,BJ McLeod_78,1,LFM,Ford,0.000,0.018,0.000


In [27]:
fuzzy_match('MartinsvilleSpeedway','Martinsville1')
nltk.edit_distance('Richmond ','Richmond1')

1

In [10]:
def add_cols_22(df, race, track = "", racenum = 0):
    df["Driver-Car"] = df["Driver"] + "_" + df["Car #"].astype(str)
    df["Points Eligible"] = [1 if row > 0 else 0 for row in df["Points"]]
    df["Team"] = df["Driver"].map(teams_22)
    df["Make"] = df["Team"].map(makes_22)
    df["Race #"] = racenum
    df["Track"] = track
    df["Track Type"] = df["Track"].map(track_types_22)
    df["Race Laps"] = laps_22[race]
    df["% Fastest Laps"] = round(df["# Fastest Laps"] / df["Race Laps"] * 100, 3)
    df["Pass Diff Per Lap"] = round(df["Pass Differential"] / df["Total Laps"], 3)
    df["Quality Passes Per Lap"] = round(df["Quality Passes"] / df["Total Laps"], 3)
    return df

In [11]:
add_cols_22(df)

TypeError: add_cols_22() missing 1 required positional argument: 'race'

In [23]:
df.head()

,Car,Driver,StartPos,MidRace,CloserPos,FinishPos,HighPos,LowPos,AvgPos,PassDiff,GreenPass,GreenPassed,QualityPasses,PercentQualityPasses,NumFastestLaps,LapsTop15,PercentLapsTop15,LapsLed,PercentLapsLed,TotalLaps,DriverRating,Pts,Date,Race,Points Eligible,Team,Make,Race #,Track,Track Type,Quality Passes Per Lap
0,5,Kyle Larson,13,3,2,1,1,25,7.1,24,83,59,63,75.90,16,183,91.5,28,14.0,200,116.3,52,20220227,AutoClub,1,Hendrick,Chevy,0,,NaN,0.315
1,3,Austin Dillon,16,12,3,2,2,31,15.6,-22,97,119,52,53.61,3,118,59.0,0,0.0,200,82.9,35,20220227,AutoClub,1,RCR,Chevy,0,,NaN,0.260
2,43,Erik Jones,2,7,6,3,1,12,4.3,13,72,59,64,88.89,23,200,100.0,18,9.0,200,122.7,52,20220227,AutoClub,1,PGMS,Chevy,0,,NaN,0.320
3,99,Daniel Suarez,15,26,4,4,1,31,14.5,-3,96,99,54,56.25,0,115,57.5,0,0.0,200,86.2,33,20220227,AutoClub,1,Trackhouse,Chevy,0,,NaN,0.270
4,22,Joey Logano,7,4,1,5,1,20,5.4,32,87,55,66,75.86,5,198,99.0,14,7.0,200,110.3,44,20220227,AutoClub,1,Penske,Ford,0,,NaN,0.330


In [97]:
df_rolling=df.groupby(['Date','Driver','Team'])[['FinishPos','Pts']].expanding().mean().reset_index()
df_rolling=df_rolling.sort_values(by=['Date','Team'])
df_rolling

,Date,Driver,Team,level_3,FinishPos,Pts
6,20220227,Bubba Wallace,23X1,18,19.0,18.0
23,20220227,Kurt Busch,23X1,7,8.0,29.0
26,20220227,Michael McDowell,FRM,30,31.0,6.0
29,20220227,Todd Gilliland,FRM,19,20.0,17.0
0,20220227,Alex Bowman,Hendrick,24,25.0,17.0
...,...,...,...,...,...,...
304,20220502,Cole Custer,SHR,347,15.0,22.0
311,20220502,Kevin Harvick,SHR,341,9.0,28.0
309,20220502,Josh Bilicki,Spire,364,32.0,0.0
316,20220502,Ross Chastain,Trackhouse,335,3.0,46.0


In [103]:
df
px.scatter(df,x='Driver',y='FinishPos',color='Race')